# Data Cleaning & Quality Audit: College Student Dataset

## Project Overview
This project simulates a real-world data cleaning task using a mock dataset of 200 college students. The dataset contains intentional quality issues to demonstrate how I identify, resolve, and document data problems.


## Objectives
- Identify and resolve data quality issues (e.g., inconsistent labels, out-of-range values)
- Apply logical validation rules (e.g., graduation year must not precede enrollment date)
- Prepare a clean, analysis-ready dataset


In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('mock_college_students_with_issues.csv')
df.head()


,student_id,first_name,last_name,gender,age,major,gpa,credits_completed,enrollment_date,graduation_year,home_state
0,S1000,Morgan,Johnson,Male,17,Biology,2.77,88,2019-11-19,2023.0,NY
1,S1001,Casey,Lee,Female,21,Biology,1.57,72,2020-04-07,NaN,CA
2,S1002,Jordan,Smith,Male,21,Sociology,2.86,67,2020-08-17,2023.0,WA
3,S1003,Casey,Smith,Male,22,Biology,3.10,100,2019-08-07,2024.0,IL
4,S1004,Casey,Brown,Male,19,Economics,2.77,97,2020-11-01,NaN,IL


## Initial Data Audit

In [2]:
# Summary statistics and structure
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   student_id         200 non-null    object 
 1   first_name         200 non-null    object 
 2   last_name          200 non-null    object 
 3   gender             200 non-null    object 
 4   age                200 non-null    int64  
 5   major              200 non-null    object 
 6   gpa                200 non-null    float64
 7   credits_completed  200 non-null    int64  
 8   enrollment_date    200 non-null    object 
 9   graduation_year    185 non-null    float64
 10  home_state         200 non-null    object 
dtypes: float64(2), int64(2), object(7)
memory usage: 17.3+ KB


,student_id,first_name,last_name,gender,age,major,gpa,credits_completed,enrollment_date,graduation_year,home_state
count,200,200,200,200,200.000000,200,200.000000,200.000000,200,185.000000,200
unique,200,5,5,7,NaN,8,NaN,NaN,187,NaN,5
top,S1000,Morgan,Smith,Female,NaN,Computer Science,NaN,NaN,2022-03-14,NaN,WA
freq,1,49,45,87,NaN,45,NaN,NaN,2,NaN,48
mean,NaN,NaN,NaN,NaN,20.815000,NaN,3.018700,61.215000,NaN,2023.740541,NaN
std,NaN,NaN,NaN,NaN,2.528387,NaN,0.429319,37.274464,NaN,1.097294,NaN
min,NaN,NaN,NaN,NaN,17.000000,NaN,1.570000,-10.000000,NaN,2020.000000,NaN
25%,NaN,NaN,NaN,NaN,19.000000,NaN,2.797500,28.000000,NaN,2023.000000,NaN
50%,NaN,NaN,NaN,NaN,21.000000,NaN,3.040000,61.500000,NaN,2024.000000,NaN
75%,NaN,NaN,NaN,NaN,23.000000,NaN,3.300000,92.000000,NaN,2024.000000,NaN


In [3]:
# Missing values
df.isnull().sum()

student_id            0
first_name            0
last_name             0
gender                0
age                   0
major                 0
gpa                   0
credits_completed     0
enrollment_date       0
graduation_year      15
home_state            0
dtype: int64

In [4]:
# Duplicate entries
df.duplicated().sum()

0

## Identifying Data Quality Issues

### Issue 1: Graduation Year Before Enrollment Date


In [5]:
df['enrollment_date'] = pd.to_datetime(df['enrollment_date'])
df[df['graduation_year'] < df['enrollment_date'].dt.year]

,student_id,first_name,last_name,gender,age,major,gpa,credits_completed,enrollment_date,graduation_year,home_state
50,S1050,Morgan,Smith,Male,17,Computer Science,3.68,57,2022-09-01,2020.0,NY
60,S1060,Taylor,Johnson,F,20,Psychology,3.38,38,2023-06-06,2022.0,TX
147,S1147,Jordan,Smith,Female,18,Computer Science,3.33,129,2023-01-26,2022.0,CA
199,S1199,Morgan,Johnson,Female,22,Economics,3.27,61,2023-05-07,2022.0,CA


### Issue 2: Inconsistent Categories in Gender and Major

In [6]:
df['gender'].value_counts()

gender
Female               87
Male                 83
Prefer Not to Say    15
Non-Binary           12
femaale               1
F                     1
fem                   1
Name: count, dtype: int64

In [7]:
df['major'].value_counts()

major
Computer Science    45
Psychology          42
Sociology           41
Economics           35
Biology             34
Comp Sci             1
computer sci         1
CS                   1
Name: count, dtype: int64

## Cleaning the Data

In [8]:
# Fix graduation year logic
df.loc[df['graduation_year'] < df['enrollment_date'].dt.year, 'graduation_year'] = np.nan

In [9]:
# Normalize gender
gender_map = {'fem': 'Female', 'F': 'Female', 'femaale': 'Female', 'M': 'Male', 'm': 'Male'}
df['gender'] = df['gender'].replace(gender_map)

In [10]:
# Normalize major
major_map = {
    'Comp Sci': 'Computer Science',
    'CS': 'Computer Science',
    'computer sci': 'Computer Science'
}
df['major'] = df['major'].replace(major_map)

In [11]:
# Fix GPA over 4.0
df.loc[df['gpa'] > 4.0, 'gpa'] = 4.0

In [12]:
# Fix negative credits
df.loc[df['credits_completed'] < 0, 'credits_completed'] = 0

## Rechecking Data After Cleaning

In [13]:
df.describe(include='all')

,student_id,first_name,last_name,gender,age,major,gpa,credits_completed,enrollment_date,graduation_year,home_state
count,200,200,200,200,200.000000,200,200.000000,200.000000,200,181.000000,200
unique,200,5,5,4,NaN,5,NaN,NaN,NaN,NaN,5
top,S1000,Morgan,Smith,Female,NaN,Computer Science,NaN,NaN,NaN,NaN,WA
freq,1,49,45,90,NaN,48,NaN,NaN,NaN,NaN,48
mean,NaN,NaN,NaN,NaN,20.815000,NaN,3.014050,61.265000,2021-08-13 12:50:24,2023.790055,NaN
min,NaN,NaN,NaN,NaN,17.000000,NaN,1.570000,0.000000,2019-08-02 00:00:00,2022.000000,NaN
25%,NaN,NaN,NaN,NaN,19.000000,NaN,2.797500,28.000000,2020-06-29 18:00:00,2023.000000,NaN
50%,NaN,NaN,NaN,NaN,21.000000,NaN,3.040000,61.500000,2021-10-09 00:00:00,2024.000000,NaN
75%,NaN,NaN,NaN,NaN,23.000000,NaN,3.300000,92.000000,2022-08-17 00:00:00,2024.000000,NaN
max,NaN,NaN,NaN,NaN,25.000000,NaN,4.000000,129.000000,2023-07-22 00:00:00,2026.000000,NaN


In [ ]:
# check for missing values
df.isnull().sum()

student_id            0
first_name            0
last_name             0
gender                0
age                   0
major                 0
gpa                   0
credits_completed     0
enrollment_date       0
graduation_year      19
home_state            0
dtype: int64

In [ ]:
# inspect rows with missing graduation year
df[df['graduation_year'].isnull()].sample(5)

,student_id,first_name,last_name,gender,age,major,gpa,credits_completed,enrollment_date,graduation_year,home_state
193,S1193,Taylor,Smith,Female,17,Psychology,2.32,113,2020-08-09,NaN,IL
50,S1050,Morgan,Smith,Male,17,Computer Science,3.68,57,2022-09-01,NaN,NY
20,S1020,Taylor,Johnson,Female,18,Biology,3.34,59,2022-05-10,NaN,TX
56,S1056,Morgan,Smith,Male,20,Sociology,2.58,99,2021-10-13,NaN,IL
1,S1001,Casey,Lee,Female,21,Biology,1.57,72,2020-04-07,NaN,CA


## Estimating Graduation Status and Year

Several records in the dataset are missing a value for `graduation_year`. These gaps may result from two primary causes:
- The student is currently enrolled and has not yet graduated
- Graduation data was omitted or entered incorrectly

To address this while maintaining data integrity and analytical flexibility:

1. A new column, `graduation_status`, was added to categorize each student as either `'Graduated'` or `'In Progress'` based on the presence of a recorded `graduation_year`.
2. An additional column, `estimated_grad_year`, was introduced to support forecasting and longitudinal analysis. For students missing a `graduation_year`, an estimate was generated by adding four years to their `enrollment_date`.

In [16]:
# Create a graduation status flag
df['graduation_status'] = df['graduation_year'].apply(
    lambda x: 'Graduated' if pd.notnull(x) else 'In Progress'
)

In [17]:
# Create an estimated graduation year (only for missing)
df['estimated_grad_year'] = df['graduation_year']  # start with the real one
mask = df['graduation_year'].isnull()
df.loc[mask, 'estimated_grad_year'] = df.loc[mask, 'enrollment_date'].dt.year + 4

In [18]:
# Preview the changes
df[['student_id', 'enrollment_date', 'graduation_year', 'graduation_status', 'estimated_grad_year']].head()

,student_id,enrollment_date,graduation_year,graduation_status,estimated_grad_year
0,S1000,2019-11-19,2023.0,Graduated,2023.0
1,S1001,2020-04-07,NaN,In Progress,2024.0
2,S1002,2020-08-17,2023.0,Graduated,2023.0
3,S1003,2019-08-07,2024.0,Graduated,2024.0
4,S1004,2020-11-01,NaN,In Progress,2024.0


## Final Formatting

To remove unnecessary decimal points caused by NaN values, the `graduation_year` and `estimated_grad_year` columns were converted to Pandas' `Int64` type. This ensures cleaner formatting while retaining support for missing values.


In [ ]:
# Format columns as integers (with support for NaN)
df['graduation_year'] = df['graduation_year'].astype('Int64')
df['estimated_grad_year'] = df['estimated_grad_year'].astype('Int64')

# Final preview of the relevant columns
df[['student_id', 'enrollment_date', 'graduation_year', 'graduation_status', 'estimated_grad_year']].head()

,student_id,enrollment_date,graduation_year,graduation_status,estimated_grad_year
0,S1000,2019-11-19,2023,Graduated,2023
1,S1001,2020-04-07,<NA>,In Progress,2024
2,S1002,2020-08-17,2023,Graduated,2023
3,S1003,2019-08-07,2024,Graduated,2024
4,S1004,2020-11-01,<NA>,In Progress,2024


## Save Cleaned Dataset

In [21]:
df.to_csv('cleaned_student_data.csv', index=False)
print("Cleaned dataset saved.")

Cleaned dataset saved.


## Summary

This notebook demonstrates a complete data cleaning and validation workflow for a mock college student dataset. The process reflects common challenges found in real-world institutional data and prepares the dataset for downstream analysis and reporting. Key steps included:

- Standardizing inconsistent categorical values such as `gender` and `major`
- Validating and correcting temporal logic issues, including `graduation_year` values that precede `enrollment_date`
- Handling missing values and correcting outliers, including invalid `gpa` and `credits_completed` entries
- Creating a `graduation_status` field to distinguish between graduated and in-progress students
- Estimating missing `graduation_year` values using a transparent, rule-based approach
- Formatting year fields using appropriate data types to ensure clean and accurate outputs

This project highlights practical skills in data auditing, preprocessing, and logic-based feature engineering using Python and Pandas.
